In [ ]:
from google.colab import drive
from google.colab import files
import requests
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pages = pd.read_csv("drive/My Drive/Diploma/second_try/Code/quarry-20220601000001-20220602000001-all-98k.csv")
pages.date = pages.date.astype(str)
pages.shape

(98629, 4)

In [ ]:
pages.head(3)

,page_title,page_id,date,count_rev
0,!!Fuck_you!!,16789001,20220601,1
1,"""A""_Street",66533585,20220601,1
2,"""Blue_Bus""_lines",31931527,20220601,1


In [ ]:
WITH r as (
  SELECT rev_page, substring(rev_timestamp, 1, 8) date
  FROM revision
  WHERE rev_timestamp BETWEEN '20220601000001' and '20220602000001'
  GROUP BY 1, 2
)

SELECT r.rev_page, r.date,  
		sum(case when f.rev_timestamp BETWEEN '20220531000001' and '20220601000001' then 1 else 0 end) same_day_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220530000001' and '20220531000001' then 1 else 0 end) prev_day_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220525000001' and '20220531000001' then 1 else 0 end) week_before_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220502000001' and '20220531000001' then 1 else 0 end) month_before_revs, 
		sum(case when f.rev_timestamp BETWEEN '20210601000001' and '20220531000001' then 1 else 0 end) year_before_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220531000001' and '20220608000001' then 1 else 0 end) week_after_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220601000001' and '20220602000001' then 1 else 0 end) day_after_revs, 
		sum(case when f.rev_timestamp BETWEEN '20220602000001' and '20220603000001' then 1 else 0 end) day_after_2_revs
				
FROM  r
	LEFT JOIN revision f on r.rev_page = f.rev_page
    						and f.rev_timestamp BETWEEN '20210601000001' and '20220608000001' 
    
  GROUP BY 1, 2

In [ ]:
# import swifter
def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], a)
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, str(i) )
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def get_revision_by_date(title, date):
  try:
    query = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&titles={title}&rvprop=ids&rvlimit=1&rvstart={date}T00%3A00%3A00.010Z"
    res = flatten_data(requests.get(query).json())['parenti']
  except:
    res = -1
  return res


def get_n_revisions(title, revision_from, revision_to):
  if revision_from==revision_to:
    return 0
  try:
    query = f"https://en.wikipedia.org/w/rest.php/v1/page/{title}/history/counts/edits?from={revision_from}&to={revision_to}"
    res = requests.get(query).json()['count']
    res +=1
  except:
    #print(title, from_, to_)
    res = -1
  return res

def substract_days(date, days):
  return datetime.strftime(datetime.strptime(date, '%Y-%m-%d') - timedelta(days=days), '%Y-%m-%d')

import datetime
from datetime import datetime, timedelta
def get_revision_features(title, date):
  same_day = get_revision_by_date(title, date)
  prev_day = get_revision_by_date(title, substract_days(date, 1))
  before_2_days = get_revision_by_date(title, substract_days(date, 2))
  week_before =  get_revision_by_date(title, substract_days(date, 7))
  month_before =  get_revision_by_date(title, substract_days(date, 30))
  year_before = get_revision_by_date(title, substract_days(date, 365))
  week_after =  get_revision_by_date(title, substract_days(date, -7))

  # print(same_day, prev_day)
  same_day_views = get_n_revisions(title, prev_day, same_day)
  prev_day_views = get_n_revisions(title, before_2_days, prev_day)
  week_before_views = get_n_revisions(title, week_before, prev_day)
  month_before_views = get_n_revisions(title, month_before, prev_day)
  year_before_views = get_n_revisions(title, year_before, prev_day)
  week_after_views = get_n_revisions(title, prev_day, week_after)

  return [title, date, same_day_views, prev_day_views, week_before_views, month_before_views, 
          year_before_views, week_after_views]

In [ ]:
get_revision_features("American_Civil_Liberties_Union", "2022-06-01")
# 1950	20220601	8	1	3	29	202	24	5

['American_Civil_Liberties_Union', '2022-06-01', 8, 1, 3, 29, 201, 24]

In [ ]:


import datetime
from datetime import datetime, timedelta
def substract_days(date, days):
  return datetime.strftime(datetime.strptime(date, '%Y-%m-%d') - timedelta(days=days), '%Y-%m-%d')

def get_revision_features(date):
  print(date, substract_days(date, 1)
        , substract_days(date, 2)
        , substract_days(date, 7)
        , substract_days(date, 30)
        , substract_days(date, 365)
        , substract_days(date, -7)
  )

get_revision_features("2022-06-01")

2022-06-01 2022-05-31 2022-05-30 2022-05-25 2022-05-02 2021-06-01 2022-06-08


In [ ]:
df = pd.read_csv("drive/My Drive/Diploma/second_try/Code/events_2022.csv")

df['title'] = df['Title link'].str.split('/', expand=True)[2]
df['date'] = df['Date'].apply(lambda x: transform_date(x))
df.head(2)

In [ ]:
a = df.swifter.apply(lambda x: get_revision_features(x['title'], x['date']), axis=1)
from google.colab import files
a.to_csv('revisions_features.csv')
files.download('views_categories_events_1_year_edits_1_day.csv')